In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/local/opt/apache-spark/libexec'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
            SparkSession
                .builder
                .appName("ErrorHandlingApp")
                .master("local[4]")
    
                .config("spark.dynamicAllocation.enabled", "false")
                .config("spark.sql.adaptive.enabled", "false")    
                .getOrCreate()
        )

sc = spark.sparkContext

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/02 11:51:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/02 11:51:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/02 11:51:12 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### Read JSON file with default Parse Mode

In [3]:
rateCodesJsonDF_modes = (
                            spark
                              .read                              
                              .json("./../files/input/RateCodes.json")
                        )

rateCodesJsonDF_modes.show(truncate=False)

+---------------+----------+-----------------------------------------+
|RateCode       |RateCodeID|_corrupt_record                          |
+---------------+----------+-----------------------------------------+
|Standard rate  |1         |null                                     |
|JFK            |2         |null                                     |
|Newark         |3         |null                                     |
|null           |null      |{"RateCodeID":},"RateCode":"Westchester"}|
|Negotiated fare|5         |null                                     |
|Group ride     |6         |null                                     |
+---------------+----------+-----------------------------------------+



### Read JSON file with Permissive mode

In [4]:
rateCodesJsonDF_modes = (
                            spark
                              .read
    
                              .option("mode", "PERMISSIVE")
    
                              .json("./../files/input/RateCodes.json")
                        )

rateCodesJsonDF_modes.show(truncate=False)

+---------------+----------+-----------------------------------------+
|RateCode       |RateCodeID|_corrupt_record                          |
+---------------+----------+-----------------------------------------+
|Standard rate  |1         |null                                     |
|JFK            |2         |null                                     |
|Newark         |3         |null                                     |
|null           |null      |{"RateCodeID":},"RateCode":"Westchester"}|
|Negotiated fare|5         |null                                     |
|Group ride     |6         |null                                     |
+---------------+----------+-----------------------------------------+



In [5]:
rateCodesJsonDF_modes = (
                            spark
                              .read
    
                              .option("mode", "PERMISSIVE")
    
                              .option("columnNameOfCorruptRecord", "CorruptData")
    
                              .json("./../files/input/RateCodes.json")
                        )

rateCodesJsonDF_modes.show(truncate=False)

+-----------------------------------------+---------------+----------+
|CorruptData                              |RateCode       |RateCodeID|
+-----------------------------------------+---------------+----------+
|null                                     |Standard rate  |1         |
|null                                     |JFK            |2         |
|null                                     |Newark         |3         |
|{"RateCodeID":},"RateCode":"Westchester"}|null           |null      |
|null                                     |Negotiated fare|5         |
|null                                     |Group ride     |6         |
+-----------------------------------------+---------------+----------+



In [6]:
rateCodesJsonDF_modes = (
                            spark
                              .read
    
                              .option("mode", "PERMISSIVE")
    
                              .option("columnNameOfCorruptRecord", "RateCodeID")
    
                              .json("./../files/input/RateCodes.json")
                        )

rateCodesJsonDF_modes.show(truncate=False)

+---------------+-----------------------------------------+
|RateCode       |RateCodeID                               |
+---------------+-----------------------------------------+
|Standard rate  |null                                     |
|JFK            |null                                     |
|Newark         |null                                     |
|null           |{"RateCodeID":},"RateCode":"Westchester"}|
|Negotiated fare|null                                     |
|Group ride     |null                                     |
+---------------+-----------------------------------------+



### Read JSON file with DropMalformed mode

In [7]:
rateCodesJsonDF_modes = (
                            spark
                              .read
    
                              .option("mode", "DROPMALFORMED")
    
                              .json("./../files/input/RateCodes.json")
                        )

rateCodesJsonDF_modes.show(truncate=False)

+---------------+----------+
|RateCode       |RateCodeID|
+---------------+----------+
|Standard rate  |1         |
|JFK            |2         |
|Newark         |3         |
|Negotiated fare|5         |
|Group ride     |6         |
+---------------+----------+



### Read JSON file with FailFast mode

In [9]:
rateCodesJsonDF_modes = (
                            spark
                              .read
    
                              .option("mode", "FAILFAST")
    
                              .json("./../files/input/RateCodes.json")
                        )

rateCodesJsonDF_modes.show(truncate=False)

23/05/02 11:51:40 ERROR Executor: Exception in task 0.0 in stage 11.0 (TID 11)
org.apache.spark.SparkException: Malformed records are detected in schema inference. Parse Mode: FAILFAST.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInSchemaInferenceError(QueryExecutionErrors.scala:1325)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema.handleJsonErrorsByParseMode(JsonInferSchema.scala:64)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema.$anonfun$infer$2(JsonInferSchema.scala:93)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.TraversableOnce.reduceLeft(TraversableOnce.scala:237)
	at scala.collection.TraversableOnce.reduceLeft$(TraversableOnce.scala:220)
	at 

Py4JJavaError: An error occurred while calling o67.json.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 11) (192.168.0.118 executor driver): org.apache.spark.SparkException: Malformed records are detected in schema inference. Parse Mode: FAILFAST.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInSchemaInferenceError(QueryExecutionErrors.scala:1325)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema.handleJsonErrorsByParseMode(JsonInferSchema.scala:64)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema.$anonfun$infer$2(JsonInferSchema.scala:93)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(I

### Read CSV file with default Permissive mode

In [11]:
rateCodesCsvDF_modes = (
                          spark
                            .read
                            .option("header", "true")
    
                            .option("mode", "PERMISSIVE")
    
                            .csv("./../files/input/RateCodes.csv")
                       )

rateCodesCsvDF_modes.show(truncate=False)

+-----------+---------------+----------+
|RateCodeID |RateCode       |IsApproved|
+-----------+---------------+----------+
|1          |Standard Rate  |Yes       |
|2          |JFK            |Yes       |
|3          |Newark         |Yes       |
|4          |null           |null      |
|Westchester|Yes            |null      |
|5          |Negotiated fare|null      |
|6          |GroupRide      |Yes       |
+-----------+---------------+----------+



### Read CSV file with DropMalformed mode

In [12]:
rateCodesCsvDF_modes = (
                          spark
                            .read
                            .option("header", "true")
    
                            .option("mode", "DROPMALFORMED")
    
                            .csv("./../files/input/RateCodes.csv")
                       )

rateCodesCsvDF_modes.show(truncate=False)

+----------+---------------+----------+
|RateCodeID|RateCode       |IsApproved|
+----------+---------------+----------+
|1         |Standard Rate  |Yes       |
|2         |JFK            |Yes       |
|3         |Newark         |Yes       |
|5         |Negotiated fare|null      |
|6         |GroupRide      |Yes       |
+----------+---------------+----------+



### Read CSV file with FailFast mode

In [13]:
rateCodesCsvDF_modes = (
                          spark
                            .read
                            .option("header", "true")
    
                            .option("mode", "FAILFAST")
    
                            .csv("./../files/input/RateCodes.csv")
                       )

rateCodesCsvDF_modes.show(truncate=False)

23/05/02 11:52:41 ERROR Executor: Exception in task 0.0 in stage 17.0 (TID 17)
org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1417)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:68)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:421)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.exe

Py4JJavaError: An error occurred while calling o99.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 17.0 failed 1 times, most recent failure: Lost task 0.0 in stage 17.0 (TID 17) (192.168.0.118 executor driver): org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1417)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:68)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:421)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apac